# Generative Adversarial Network

Basado en https://www.tensorflow.org/tutorials/generative/dcgan

Algunos trucos: https://towardsdatascience.com/gan-by-example-using-keras-on-tensorflow-backend-1a6d515a60d0

Para usar imágenes RGB he consultado https://machinelearningmastery.com/how-to-develop-a-generative-adversarial-network-for-a-cifar-10-small-object-photographs-from-scratch/

https://www.tensorflow.org/js/tutorials/conversion/import_keras

https://blog.tensorflow.org/2018/07/train-model-in-tfkeras-with-colab-and-run-in-browser-tensorflowjs.html

https://medium.com/tensorflow/train-on-google-colab-and-run-on-the-browser-a-case-study-8a45f9b1474e


In [ ]:
!pip install tensorflowjs 

import tensorflow as tf
import tensorflowjs as tfjs

import glob
import imageio
import matplotlib.pyplot as plt
import numpy as np
import os
from PIL import Image, ImageDraw
from tensorflow.keras import layers
import time
from google.colab import files as GCfiles

from IPython.display import display, clear_output
from ipywidgets import Output

%config InlineBackend.figure_format='retina'

In [ ]:
# Número de elementos que va a tener nuestro conjunto de datos de entrenamiento
BUFFER_SIZE = 25000

# Tamaño del batch
BATCH_SIZE = 256

# Número de entradas del generador
INPUTS = 100

# Ancho y alto de cada imagen 
w, h = 64, 64

# Cuánto vamos a entrenar la red
EPOCHS = 100

# El número de ejemplos que vamos a mostrar durante el entrenamiento 
NUM_EXAMPLES = 18

# La semilla para esos ejemplos
seed = np.random.normal(size=(NUM_EXAMPLES, INPUTS))

### Crear el dataset

In [ ]:
# El objetivo es rellenar un array de tamaño (BUFFER_SIZE,w,h,3) con todas 
# las imágenes que generamos. El array está normalizado entre -1 y 1 (0..255)

train_images = np.empty((BUFFER_SIZE,w,h,3))

colores = [(23, 63, 95), (32, 99, 155), (60, 174, 163), (246, 213, 92), (237, 85, 59)]

for f in range(BUFFER_SIZE):

  if f%1000==0:
    print(f'{f}/{BUFFER_SIZE} elementos')

  img = Image.new('RGB', (w, h), color=colores[0])
  canvas = ImageDraw.Draw(img) 
  
  coords = np.random.randint(-11,w-12,size=2)
  coords = np.append(coords, coords+24)
  canvas.ellipse(coords.tolist(), fill=colores[np.random.randint(4)+1], outline=None)
  
  for g in range(5): 
    coords = np.random.randint(-1,8,size=2)*8
    coords = np.append(coords, coords+10)
    canvas.rectangle(coords.tolist(), fill=colores[np.random.randint(4)+1], outline=None)
  
  item = np.array(img.getdata())
  item = (item-127.5) / 127.5
  item = np.reshape(item,(w, h, 3))
  train_images[f]=item

fig = plt.figure(figsize=(8, 4))
for i in range(NUM_EXAMPLES):
  plt.subplot(3, 6, i+1)
  plt.imshow(train_images[i, :, :] * .5 + .5)
  plt.axis('off')
plt.show()


In [ ]:
train_dataset = tf.data.Dataset.from_tensor_slices(train_images.astype('float32')).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)

### Modelos

In [ ]:
# El generador recibe INPUTS valores aleatorios entre -1 y 1 y devuelve
# un array de W x H (la imagen generada)

def make_generator_model():
  
  model = tf.keras.Sequential()

  # model.add(layers.Dense(8*8*256, use_bias=False, input_shape=(INPUTS,)))
  model.add(layers.Dense(4*4*128, use_bias=False, input_shape=(INPUTS,)))
  model.add(layers.BatchNormalization())
  model.add(layers.LeakyReLU())

  model.add(layers.Reshape((4, 4, 128)))

  model.add(layers.Conv2DTranspose(128, (4, 4), strides=(2, 2), padding='same', use_bias=False))
  model.add(layers.BatchNormalization())
  model.add(layers.LeakyReLU())

  model.add(layers.Conv2DTranspose(128, (4, 4), strides=(2, 2), padding='same', use_bias=False))
  model.add(layers.BatchNormalization())
  model.add(layers.LeakyReLU())
  
  model.add(layers.Conv2DTranspose(128, (4, 4), strides=(2, 2), padding='same', use_bias=False))
  model.add(layers.BatchNormalization())
  model.add(layers.LeakyReLU())

  model.add(layers.Conv2DTranspose(3, (4, 4), strides=(2, 2), padding='same', activation='tanh'))
  
  # model.add(layers.Conv2DTranspose(128, (4, 4), strides=(2, 2), padding='same', use_bias=False))
  # model.add(layers.BatchNormalization())
  # model.add(layers.LeakyReLU())
  # model.add(layers.Conv2D(3, (3,3), padding='same', activation='tanh'))



  # assert model.output_shape == (None, w, h, 3)

  return model

In [ ]:
# El discriminador recibe un array de W x H (la imagen generada) y devuelve 
# un 0 (es generada) o un 1 (es verdadera)

def make_discriminator_model():
  model = tf.keras.Sequential()

  model.add(layers.Conv2D(64, (5, 5), 
                          strides=(2, 2), padding='same',
                          input_shape=[w, h, 3]))
  model.add(layers.LeakyReLU())
  model.add(layers.Dropout(0.3))

  model.add(layers.Conv2D(128, (5, 5), 
                          strides=(2, 2), padding='same'))
  model.add(layers.LeakyReLU())
  model.add(layers.Dropout(0.3))

  model.add(layers.Conv2D(256, (5, 5), 
                          strides=(2, 2), padding='same'))
  model.add(layers.LeakyReLU())
  model.add(layers.Dropout(0.3))

  model.add(layers.Flatten())
  model.add(layers.Dense(1))

  return model

In [ ]:
# Instanciamos los modelos
generator = make_generator_model()
discriminator = make_discriminator_model()

# Instanciamos los optimizadores
generator_optimizer = tf.keras.optimizers.Adam(1e-4)
discriminator_optimizer = tf.keras.optimizers.Adam(1e-4)

# Definimos el directorio donde se van a grabar los checkpoints
checkpoint_dir = './'
checkpoint_prefix = os.path.join(checkpoint_dir, 'GAN-ckpt')
checkpoint = tf.train.Checkpoint(generator_optimizer=generator_optimizer,
                                 discriminator_optimizer=discriminator_optimizer,
                                 generator=generator,
                                 discriminator=discriminator)

In [ ]:
# Como el problema es clasificación binaria, usamos esta función de error
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

# El discriminador debería devolver un cero cuando la imagen ha sido generada por 
# el generador y un 1 si viene del training set.
def discriminator_loss(real_output, fake_output):
  real_loss = cross_entropy(tf.ones_like(real_output), real_output)
  fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
  total_loss = real_loss + fake_loss
  return total_loss

# Las imágenes generadas por el generador siempre deberían conseguir 
# un 1 cuando se presentan al discriminador
def generator_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output), fake_output)

In [ ]:
# noise = tf.random.normal([1, INPUTS])
# generated_image = generator(noise, training=False)
# plt.imshow(generated_image[0, :, :] *0.5 + 0.5)

# decision = discriminator(generated_image)
# print (decision)

# generator.summary()
# discriminator.summary()

In [ ]:
# En cada paso, calculamos la salida del discriminador para las imágenes del
# dataset, para las imágenes falsas, calculamos los gradientes y los opimizamos.
@tf.function
def train_step(real_images):

  noise = tf.random.normal([BATCH_SIZE, INPUTS])
  
  with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:

    # Salida del discriminador cuando se le presentan imágenes reales. 
    # Deberían ser todo unos.
    real_output = discriminator(real_images, training=True) 
    
    # Salida del discriminador cuando se le presentan imágenes falsas.
    # Deberían ser todo ceros.
    fake_images = generator(noise, training=True)
    fake_output = discriminator(fake_images, training=True)

    # Se calculan las pérdidas
    gen_loss = generator_loss(fake_output)
    disc_loss = discriminator_loss(real_output, fake_output)

  gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
  gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

  generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
  discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))

  return gen_loss, disc_loss

# Lo importante es iterar sobre el dataset tantas veces como epochs hayamos
# indicado. El resto es para sacar información adicional y grabar de vez en 
# cuando.

# La versión mínima sería
# def train(dataset, epochs):
#   for epoch in range(epochs):
#     print(epoch)
#     for image_batch in dataset:
#       _, _ = train_step(image_batch)

def train(dataset, epochs):

  list_gen_loss = []
  list_disc_loss = [] 
  
  examples = Output()
  display(examples)

  loss_plot = Output()
  display(loss_plot)

  for epoch in range(epochs):
    start = time.time()

    total_gen_loss = 0
    total_disc_loss = 0
    steps = 0
    for image_batch in dataset:
      gen_loss, disc_loss = train_step(image_batch)
      total_gen_loss += gen_loss
      total_disc_loss += disc_loss
      steps +=1

    list_gen_loss.append(total_gen_loss/steps)
    list_disc_loss.append(total_disc_loss/steps)

    if (epoch + 1) % 100 == 0:
      checkpoint.save(file_prefix = checkpoint_prefix)

    predictions = generator(seed, training=False)

    fig = plt.figure(figsize=(8, 4))
    for i in range(predictions.shape[0]):
      plt.subplot(3, 6, i+1)
      plt.imshow(predictions[i, :, :] * .5 + .5)
      plt.axis('off')

    with examples:
      clear_output(wait=True)
      plt.show()
      print ('\n\nepoch {}: gen_loss={:.2f} disc_loss={:.2f} time={:.2f} sec'.format(epoch + 1, 
                                                                    list_gen_loss[-1],
                                                                    list_disc_loss[-1],
                                                                    time.time()-start))

    if epoch%20 == 0:
      plt.plot(list_gen_loss)
      plt.plot(list_disc_loss)
      plt.legend(['generator', 'discriminator'])
      with loss_plot:
        clear_output(wait=True)
        plt.show()

In [ ]:
# Entrenamiento
# Es normal que el error del generador vaya creciendo con el tiempo
train(train_dataset, EPOCHS)

In [ ]:
# Generamos un modelo compatible con javascript y lo descargamos.
tfjs.converters.save_keras_model(generator, './generator/')
!zip -r generator.zip generator 
GCfiles.download('generator.zip') 

In [ ]:
# Recuperamos del disco los pesos que se han grabado automáticamente.
# Esta línea no pinta nada aquí, es solo un ejemplo de cómo hacerlo.
checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))

### Resultados

In [ ]:
INTERPOLATION_STEPS = 12

point = np.random.normal(size=INPUTS)
latent_seed = np.stack([point for f in range(INTERPOLATION_STEPS)])

delta = np.linspace(-1,1,INTERPOLATION_STEPS)

for f in range(15):
  dim = np.random.randint(INPUTS)
  latent_seed[:,dim] += delta

start = time.time()
generated_images = generator(latent_seed, training=False)
print ('time={:.2f} sec'.format(time.time()-start))

fig = plt.figure(figsize=(20, 4))

for i, coords in enumerate(latent_seed):
  plt.subplot(1, 13, i+1)
  plt.imshow(generated_images[i, :, :] * .5 + .5)
  plt.axis('off')
plt.show()


In [ ]:
latent_seed = tf.random.normal([1, INPUTS])
generated_image = generator(latent_seed, training=False)
plt.imshow(generated_image[0, :, :] * .5 + .5)